# Build a ReAct AI Agent

Reason+Act agent that iteratively reasons and gathers information from external tools before providing an answer.

In [ ]:
# !pip install openai==0.28.1 --quiet

In [1]:
import re
import httpx
import openai

In [3]:

openai.api_key = "sk-proj-ORcaYv7A.."


## Define Prompt for our usecase

The prompt tells the LLM how to approach a problem. This prompt is a simple example and most definitely for demonstration purposes, only. 

In [4]:
prompt = """
You run in a loop of Thought, Action, Observation, Answer.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you.
Observation will be the result of running those actions.
Answer will be the result of analysing the Observation

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

wikipedia:
e.g. wikipedia: Django
Returns a summary from searching Wikipedia

Always look things up on Wikipedia if you have the opportunity to do so.

Example session:

Question: What is the capital of France?
Thought: I should look up France on Wikipedia
Action: wikipedia: France

You should then call the appropriate action and determine the answer from the result

You then output:

Answer: The capital of France is Paris
""".strip()

## Implement a Chatbot

creating a client and then defining a Python class that will implement a chatbot

In [5]:
class ChatBot:
    def __init__(self, system=""):
        self.system = system
        self.messages = []

    def __call__(self, message):
        self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result

    def execute(self):
        conversation = self.system + "\n".join([f"{msg['role']}: {msg['content']}" for msg in self.messages])
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": self.system},
                *self.messages
            ],
            max_tokens=1000,
            temperature=0
        )
        result = response.choices[0].message['content']
        return result

## Define a Query
The next step is to define a query() function that uses an instantiation of the chatbot. The query function implements a loop that continues until there are no more actions (or we've reached the maximum number of iterations). 

In [6]:
action_re = re.compile(r'^Action: (\w+): (.*)$')

def query(question, max_turns=5, prompt=prompt):
    i = 0
    bot = ChatBot(system=prompt)
    next_prompt = question
    while i < max_turns:
        i += 1
        result = bot(next_prompt)
        print(result)
        actions = [action_re.match(a) for a in result.split('\n') if action_re.match(a)]
        if actions:
            # There is an action to run
            action, action_input = actions[0].groups()
            if action not in known_actions:
                raise Exception("Unknown action: {}: {}".format(action, action_input))
            print(" -- running {} {}".format(action, action_input))
            observation = known_actions[action](action_input)
            print("Observation:", observation)
            next_prompt = "Observation: {}".format(observation)
        else:
            return bot.messages

## Define the Action Functions (tools)
wikipedia and eval and we store references to those in a dictionary.

In [7]:
def wikipedia(q):
    return httpx.get("https://en.wikipedia.org/w/api.php", params={
        "action": "query",
        "list": "search",
        "srsearch": q,
        "format": "json"
    }).json()["query"]["search"][0]["snippet"]

def calculate(what):
    return eval(what)

known_actions = {
    "wikipedia": wikipedia,
    "calculate": calculate
}

Next is a utility function that prints out the last message to have been generated by the bot.

In [8]:
def get_last_message(bot):
    for m in bot.messages[-1]['content'][0].text.split('\n'):
        print(m)

## Test the Agent

In [9]:
bot_instance = ChatBot(system=prompt)
query("What is 20 * 15", prompt=prompt)

APIRemovedInV1: 

You tried to access openai.ChatCompletion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742


In [28]:
bot_instance = ChatBot(system=prompt)
query("What languages are spoken in Spain that are also spoken in France")

Thought: I should look up the languages spoken in Spain and France on Wikipedia to find out which ones are spoken in both countries.

Action: wikipedia: Languages of Spain

 -- running wikipedia Languages of Spain
Observation: The majority <span class="searchmatch">of</span> <span class="searchmatch">languages</span> <span class="searchmatch">of</span> <span class="searchmatch">Spain</span> belong to the Romance <span class="searchmatch">language</span> family, <span class="searchmatch">of</span> which <span class="searchmatch">Spanish</span> is the only one with official status in the whole country.
Answer: The main language spoken in both Spain and France is Spanish. While there are other regional languages spoken in Spain, such as Catalan, Galician, and Basque, they are not as widely spoken in France.


[{'role': 'user',
  'content': 'What languages are spoken in Spain that are also spoken in France'},
 {'role': 'assistant',
  'content': 'Thought: I should look up the languages spoken in Spain and France on Wikipedia to find out which ones are spoken in both countries.\n\nAction: wikipedia: Languages of Spain\n'},
 {'role': 'user',
  'content': 'Observation: The majority <span class="searchmatch">of</span> <span class="searchmatch">languages</span> <span class="searchmatch">of</span> <span class="searchmatch">Spain</span> belong to the Romance <span class="searchmatch">language</span> family, <span class="searchmatch">of</span> which <span class="searchmatch">Spanish</span> is the only one with official status in the whole country.'},
 {'role': 'assistant',
  'content': 'Answer: The main language spoken in both Spain and France is Spanish. While there are other regional languages spoken in Spain, such as Catalan, Galician, and Basque, they are not as widely spoken in France.'}]

In [10]:
bot_instance = ChatBot(system=prompt)
query("Who are some billionaires from India that are also ranked as one of the richest people list in the world")

APIRemovedInV1: 

You tried to access openai.ChatCompletion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742


In [11]:
bot_instance = ChatBot(system=prompt)
query("What is the longest river in the World?")

APIRemovedInV1: 

You tried to access openai.ChatCompletion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742
